In [1]:
%maven ai.djl:basicdataset:0.23.0
%maven ai.djl.mxnet:mxnet-engine:0.23.0
%maven org.slf4j:slf4j-simple:1.7.36
%maven pt.mleiria:deep-learning-app:1.1-SNAPSHOT
%maven org.postgresql:postgresql:42.7.7
%maven com.zaxxer:HikariCP:7.0.2
%maven org.tribuo:tribuo-core:4.3.2
%maven org.tribuo:tribuo-data:4.3.2
%maven org.tribuo:tribuo-math:4.3.2
%maven org.tribuo:tribuo-regression-core:4.3.2
%maven org.tribuo:tribuo-regression-sgd:4.3.2
%maven org.tribuo:tribuo-regression-tree:4.3.2
%maven org.tribuo:tribuo-regression-xgboost:4.3.2

In [2]:
import pt.mleiria.*;
import pt.mleiria.config.*;
import pt.mleiria.core.*;
import pt.mleiria.data.importer.*;
import pt.mleiria.data.importer.config.*;
import pt.mleiria.data.importer.start.*;
import pt.mleiria.db.*;
import pt.mleiria.vo.*;
import com.zaxxer.hikari.*;
import tech.tablesaw.api.*;

In [3]:
import org.tribuo.*;
import org.tribuo.data.csv.CSVLoader;
import org.tribuo.datasource.ListDataSource;
import org.tribuo.evaluation.TrainTestSplitter;
import org.tribuo.math.optimisers.*;
import org.tribuo.regression.*;
import org.tribuo.regression.evaluation.*;
import org.tribuo.regression.sgd.RegressionObjective;
import org.tribuo.regression.sgd.linear.LinearSGDTrainer;
import org.tribuo.regression.sgd.objectives.SquaredLoss;
import org.tribuo.regression.rtree.CARTRegressionTrainer;
import org.tribuo.regression.xgboost.XGBoostRegressionTrainer;
import org.tribuo.util.Util;

In [4]:
String workDir = "/home/manuel/Downloads/Databases/ThreeBodyProblem/X_test/processed_data/";

In [5]:
Table trainData = Table.read().file(workDir + "train_data.csv");
trainData

                                                                                                                                                                                                                                                                                                                                                                                                                            
     t      |          x_1          |         y_1          |         v_x_1         |         v_y_1         |          x_2          |          y_2          |         v_x_2         |         v_y_2         |          x_3          |          y_3          |         v_x_3         |        v_y_3         |  x0_1  |  y0_1  |         x0_2          |         y0_2         |         x0_3          |         y0_3          |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
Table xTrainTable = trainData.select("t","x0_1","y0_1","x0_2","y0_2","x0_3","y0_3");
Table yTrainTable = trainData.select("x_1","y_1","x_2","y_2","x_3","y_3");

In [7]:
xTrainTable

                                                                                                                              
     t      |  x0_1  |  y0_1  |         x0_2          |         y0_2         |         x0_3          |         y0_3          |
------------------------------------------------------------------------------------------------------------------------------
         0  |     1  |     0  |  -0.1141920208416451  |  0.8160119370682374  |  -0.8858079791583549  |  -0.8160119370682374  |
 0.0390625  |     1  |     0  |  -0.1141920208416451  |  0.8160119370682374  |  -0.8858079791583549  |  -0.8160119370682374  |
  0.078125  |     1  |     0  |  -0.1141920208416451  |  0.8160119370682374  |  -0.8858079791583549  |  -0.8160119370682374  |
  0.117188  |     1  |     0  |  -0.1141920208416451  |  0.8160119370682374  |  -0.8858079791583549  |  -0.8160119370682374  |
   0.15625  |     1  |     0  |  -0.1141920208416451  |  0.8160119370682374  |  -0.8858079791583549  |  -0.8160

In [8]:
yTrainTable

                                                                                                                                              
         x_1          |         y_1          |          x_2          |          y_2          |          x_3          |          y_3          |
----------------------------------------------------------------------------------------------------------------------------------------------
                   1  |                   0  |  -0.1141920208416451  |   0.8160119370682374  |  -0.8858079791583549  |  -0.8160119370682374  |
  0.9995113498723968  |  0.0001646301325938  |  -0.1139693126265833  |   0.8155638643857739  |  -0.8855420372458137  |  -0.8157284945183645  |
  0.9980442715440148  |   0.000658993626249  |  -0.1133005010004792  |   0.8142187501676256  |  -0.8847437705435363  |  -0.8148777437938757  |
  0.9955953659339756  |  0.0014845169628306  |   -0.112183513764467  |    0.811973895343652  |  -0.8834118521695098  |  -0.8134584123064789  |

RuntimeException: jdk.jshell.EvalException: Exception occurred while running line magic 'load': null